In [1]:
import pandas as pd

df3 = pd.read_sql_query("SELECT * FROM gvs.vw_ml_land_only_glm_ready_v3;", engine)

NameError: name 'engine' is not defined

In [2]:
from dotenv import load_dotenv
from pathlib import Path
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus

load_dotenv(Path(r"C:\OneDrive\gvs-kyiv\env\.env"))

host = os.getenv("PGHOST", "localhost")
port = int(os.getenv("PGPORT", "5432"))
user = os.getenv("PGUSER")
pwd  = os.getenv("PGPASSWORD")
db   = os.getenv("PGDATABASE")

url = f"postgresql+psycopg2://{user}:{quote_plus(pwd)}@{host}:{port}/{db}"
engine = create_engine(url, pool_pre_ping=True)

In [3]:
import pandas as pd

df3 = pd.read_sql_query("SELECT * FROM gvs.vw_ml_land_only_glm_ready_v3;", engine)

In [5]:
df3.shape, df3.columns.tolist()

((52, 16),
 ['spatial_unit_id',
  'y_ln_uah_m2',
  'x_ln_area',
  'lon',
  'lat',
  'x_ln_dist_metro',
  'x_ln_dist_school',
  'x_ln_dist_kindergarten',
  'x_ln_dist_park',
  'x_ln_dist_water',
  'x_ln_dist_cemetery',
  'x_ln_dist_highway',
  'x_ln_dist_powerline',
  'gp_band_1',
  'gp_band_2',
  'gp_band_3'])

In [6]:
pd.read_sql_query("""
select column_name
from information_schema.columns
where table_schema='gvs' and table_name='vw_ml_land_only_glm_ready_v2'
order by ordinal_position;
""", engine)

,column_name
0,spatial_unit_id
1,y_ln_uah_m2
2,lon
3,lat
4,x_ln_area
5,x_ln_dist_metro
6,x_ln_dist_school
7,x_ln_dist_kindergarten
8,x_ln_dist_park
9,x_ln_dist_water


In [7]:
base_cols = [
    "x_ln_area",
    "lon", "lat",
    "x_ln_dist_metro",
    "x_ln_dist_school",
    "x_ln_dist_kindergarten",
    "x_ln_dist_park",
    "x_ln_dist_water",
    "x_ln_dist_cemetery",
    "x_ln_dist_highway",
    "x_ln_dist_powerline",
    "gp_band_1", "gp_band_2", "gp_band_3",
]


In [8]:
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score

y = df3["y_ln_uah_m2"].astype(float)
X_base = df3[base_cols].astype(float)
X_base = sm.add_constant(X_base)

m_base = sm.OLS(y, X_base).fit(cov_type="HC3")
print(m_base.summary())


                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     11.93
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           9.59e-10
Time:                        23:40:04   Log-Likelihood:                -25.809
No. Observations:                  52   AIC:                             81.62
Df Residuals:                      37   BIC:                             110.9
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -37

In [9]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()
y_true, y_pred = [], []

for tr, te in loo.split(X_base):
    m = sm.OLS(y.iloc[tr], X_base.iloc[tr]).fit()
    y_hat = m.predict(X_base.iloc[te])
    y_true.append(y.iloc[te].values[0])
    y_pred.append(y_hat.values[0])

y_true = np.array(y_true)
y_pred = np.array(y_pred)

r2_log_base = r2_score(y_true, y_pred)
mae_base = mean_absolute_error(np.exp(y_true), np.exp(y_pred))
mape_base = np.mean(np.abs((np.exp(y_true) - np.exp(y_pred)) / np.exp(y_true))) * 100

r2_log_base, mae_base, mape_base


(0.41766724704854974, 3341.991557910467, np.float64(43.7652935952949))

In [10]:
osm_cols = [c for c in df3.columns if c.startswith("x_ln_osm_")]
cnt_cols = [c for c in df3.columns if c.startswith("x_ln_cnt500_")]

full_cols = base_cols + osm_cols + cnt_cols


In [11]:
X_full = df3[full_cols].astype(float)
X_full = sm.add_constant(X_full)

m_full = sm.OLS(y, X_full).fit(cov_type="HC3")
print(m_full.summary())


                            OLS Regression Results                            
Dep. Variable:            y_ln_uah_m2   R-squared:                       0.737
Model:                            OLS   Adj. R-squared:                  0.637
Method:                 Least Squares   F-statistic:                     11.93
Date:                Sun, 18 Jan 2026   Prob (F-statistic):           9.59e-10
Time:                        23:41:38   Log-Likelihood:                -25.809
No. Observations:                  52   AIC:                             81.62
Df Residuals:                      37   BIC:                             110.9
Df Model:                          14                                         
Covariance Type:                  HC3                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                    -37

In [12]:
df3 = pd.read_sql_query("SELECT * FROM gvs.vw_ml_land_only_glm_ready_v3;", engine)
print(df3.shape)
print(df3.columns.tolist())


(52, 16)
['spatial_unit_id', 'y_ln_uah_m2', 'x_ln_area', 'lon', 'lat', 'x_ln_dist_metro', 'x_ln_dist_school', 'x_ln_dist_kindergarten', 'x_ln_dist_park', 'x_ln_dist_water', 'x_ln_dist_cemetery', 'x_ln_dist_highway', 'x_ln_dist_powerline', 'gp_band_1', 'gp_band_2', 'gp_band_3']
